In [ ]:
import os
import json
import sqlite3
from tqdm import tqdm

# 1. Dossier contenant les fichiers JSON
folder_path = "./data"
db_path = "../sb_python_sql.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 2. Suppression de la base de données existante (en requête SQL)
cursor.execute("DROP TABLE IF EXISTS PlaylistTrack")
cursor.execute("DROP TABLE IF EXISTS Track")
cursor.execute("DROP TABLE IF EXISTS Artist")
cursor.execute("DROP TABLE IF EXISTS Album")
cursor.execute("DROP TABLE IF EXISTS Playlist")
conn.commit()

# 3. Création des tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Playlist (
    pid INTEGER PRIMARY KEY,
    name VARCHAR,
    collaborative BOOLEAN,
    modified_at BIGINT,
    num_tracks INTEGER,
    num_albums INTEGER,
    num_followers INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Track (
    track_uri VARCHAR PRIMARY KEY,
    track_name VARCHAR,
    duration_ms INTEGER,
    artist_uri VARCHAR,
    album_uri VARCHAR,
    FOREIGN KEY (artist_uri) REFERENCES Artist(artist_uri),
    FOREIGN KEY (album_uri) REFERENCES Album(album_uri)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Artist (
    artist_uri VARCHAR PRIMARY KEY,
    artist_name VARCHAR
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Album (
    album_uri VARCHAR PRIMARY KEY,
    album_name VARCHAR
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS PlaylistTrack (
    playlist_id INTEGER,
    track_uri VARCHAR,
    position INTEGER,
    PRIMARY KEY (playlist_id, track_uri, position),
    FOREIGN KEY (playlist_id) REFERENCES Playlist(pid),
    FOREIGN KEY (track_uri) REFERENCES Track(track_uri)
)
''')

conn.commit()

# 4. Traitement des fichiers JSON
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
print(f"{len(json_files)} fichiers trouvés dans {folder_path}")

playlists_data = []
artists_data = set()
albums_data = set()
tracks_data = set()
playlisttracks_data = []

for filename in tqdm(json_files, desc="Importation JSON → SQL"):
    file_path = os.path.join(folder_path, filename)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        playlists = data.get("playlists")
        if playlists is None:
            if isinstance(data, list):
                playlists = data
            elif isinstance(data, dict) and "pid" in data:
                playlists = [data]
            else:
                print(f"Format inattendu dans le fichier : {filename}")
                continue

        for playlist in playlists:
            pid = playlist.get("pid")
            if pid is None:
                continue

            playlists_data.append((
                pid,
                playlist.get("name"),
                playlist.get("collaborative") == "true",
                playlist.get("modified_at"),
                playlist.get("num_tracks"),
                playlist.get("num_albums"),
                playlist.get("num_followers")
            ))

            for track in playlist.get("tracks", []):
                artists_data.add((
                    track.get("artist_uri"),
                    track.get("artist_name")
                ))
                albums_data.add((
                    track.get("album_uri"),
                    track.get("album_name")
                ))
                tracks_data.add((
                    track.get("track_uri"),
                    track.get("track_name"),
                    track.get("duration_ms"),
                    track.get("artist_uri"),
                    track.get("album_uri")
                ))
                playlisttracks_data.append((
                    pid,
                    track.get("track_uri"),
                    track.get("pos")
                ))
    except Exception as e:
        print(f"Erreur avec {filename} : {e}")

# Insertion groupée
try:
    cursor.execute("PRAGMA foreign_keys = OFF")
    cursor.executemany("""
        INSERT OR IGNORE INTO Playlist (pid, name, collaborative, modified_at, num_tracks, num_albums, num_followers)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, playlists_data)
    cursor.executemany("""
        INSERT OR IGNORE INTO Artist (artist_uri, artist_name)
        VALUES (?, ?)
    """, list(artists_data))
    cursor.executemany("""
        INSERT OR IGNORE INTO Album (album_uri, album_name)
        VALUES (?, ?)
    """, list(albums_data))
    cursor.executemany("""
        INSERT OR IGNORE INTO Track (track_uri, track_name, duration_ms, artist_uri, album_uri)
        VALUES (?, ?, ?, ?, ?)
    """, list(tracks_data))
    cursor.executemany("""
        INSERT OR IGNORE INTO PlaylistTrack (playlist_id, track_uri, position)
        VALUES (?, ?, ?)
    """, playlisttracks_data)
    conn.commit()
finally:
    cursor.execute("PRAGMA foreign_keys = ON")

# 5. Statistiques
cursor.execute("SELECT COUNT(*) FROM Playlist")
print("Playlists :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Track")
print("Tracks :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Artist")
print("Artists :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Album")
print("Albums :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM PlaylistTrack")
print("Relations Playlist-Track :", cursor.fetchone()[0])

print("/n🎉 Import terminé avec succès !")
conn.close()


1 fichiers trouvés dans ./json


Importation JSON → SQL: 100%|██████████| 1/1 [00:00<00:00, 292.84it/s]

Playlists : 1
Tracks : 51
Artists : 37
Albums : 47
Relations Playlist-Track : 52
/n🎉 Import terminé avec succès !
